In [19]:
def high_ord_func(x: int , func) -> int:
    return func(x)+x

func = lambda a : a**2 
print(type(func))
print(high_ord_func(2, func)) 


<class 'function'>
6


In [3]:
def out_f():
    def in_f():
        print("I'm in the inner function")
    return in_f

f = out_f()
print(type(f))
f()

<class 'function'>
I'm in the inner function


In [8]:
def wrap(func):
    def wrapper():
        print("started")
        func()
        print("ended")
    return wrapper

def a_function():
    print("I'm a function")


In [9]:
print("first way:outer_function(a_function)() ")
wrap(a_function)()
print("\nsecond way: variable ")
f_wrapped = wrap(a_function)
f_wrapped()

first way:outer_function(a_function)() 
started
I'm a function
ended

second way: variable 
started
I'm a function
ended


In [12]:
@wrap
def a_function():
    print("I'm a new function")

# Equivalent to:
#    a_function = wrap(a_function)

a_function()

started
I'm a new function
ended


In [15]:
@wrap
def param_function(num: int):
    print(f"{num} is a number")
param_function(3) #We should get a TypeError
# param_function() #We should get another TypeError

TypeError: wrap.<locals>.wrapper() takes 0 positional arguments but 1 was given

In [16]:
def my_sum(*args):
    result = 0
    for x in args:
        result += x
    return result

print(my_sum(1,2,3,4,5))


15


In [17]:
def f(**kwargs):
    print(kwargs)
    print(type(kwargs))
    for key, val in kwargs.items():
        print(key, '->', val)

f(e=1, r=2,j=3)


{'e': 1, 'r': 2, 'j': 3}
<class 'dict'>
e -> 1
r -> 2
j -> 3


In [18]:
def wrap_with_params(func):
    def wrapper(*args , **kwargs):
        print("started")
        func(*args, **kwargs)
        print("ended")
    return wrapper

@wrap_with_params
def param_function(num: int):
    print(f"{num} is a number")
param_function(3)

started
3 is a number
ended


In [20]:
def wrap_with_return(func):
    def wrapper(*args , **kwargs):
        print("enter your name with no capital letters:")
        return_val = func(*args, **kwargs)
        #do somthing
        return return_val
    return wrapper

@wrap_with_return
def change_name_to_upper():
    """Changes the first character of the name to upper case """
    first_name= input("Your first name is: ")
    last_name= input("Your last name is: ")
    fname_list=list(first_name)
    lname_list=list(last_name)
    fname_list[0]= fname_list[0].upper()
    lname_list[0]= lname_list[0].upper()
    return "".join(fname_list)+" "+"".join(lname_list)
print(change_name_to_upper())

enter your name with no capital letters:
Erel Segal


In [1]:
import logging, sys
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def my_logger(orig_func):
    def wrapper(*args, **kwargs):
        logging.info(
            f'Running function {orig_func.__name__} with args {args}, and kwargs: {kwargs}')
        return orig_func(*args, **kwargs)
    return wrapper

In [4]:
@my_logger
def test1(a, b):
    print(f'{a}+{b}={a+b}')

test1(5, 7)
test1(5, b=7)

INFO:root:Running function test1 with args (5, 7), and kwargs: {}
INFO:root:Running function test1 with args (5,), and kwargs: {'b': 7}


5+7=12
5+7=12


In [5]:

def my_timer(orig_func):
    import time
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print(f'{orig_func.__name__} ran in: {t2} sec')
        return result

    return wrapper

In [6]:

@my_timer
def test2(a, b):
    print(f'{a}+{b}={a+b}')

test2(5, 7)

5+7=12
test1 ran in: 0.0 sec


In [8]:
@my_logger
@my_timer
def test3(a, b):
    print(f'{a}+{b}={a+b}')

test3(5, 7)  # Wrong result: "running function wrapper"

INFO:root:Running function wrapper with args (5, 7), and kwargs: {}


5+7=12
test3 ran in: 0.0 sec


In [10]:
# Decorators
from functools import wraps

def my_logger(orig_func):
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
            f'Running function {orig_func.__name__} with args {args}, and kwargs: {kwargs}')
        return orig_func(*args, **kwargs)
    return wrapper

def my_timer(orig_func):
    import time
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print(f'{orig_func.__name__} ran in: {t2} sec')
        return result
    return wrapper


@my_logger
@my_timer
def test4(a, b):
    print(f'{a}+{b}={a+b}')

test4(5, 7)
test4(5, b=7)

INFO:root:Running function test4 with args (5, 7), and kwargs: {}
INFO:root:Running function test4 with args (5,), and kwargs: {'b': 7}


5+7=12
test4 ran in: 0.0 sec
5+7=12
test4 ran in: 0.0 sec


In [11]:
def mozzarella(pizza) -> float:
    #@wraps()
    def wrapper(*args, **kwargs):
        return pizza()+3.5
    return wrapper

def onion(pizza) -> float:
    #@wraps()
    def wrapper(*args, **kwargs):
        return pizza()+2.25
    return wrapper

def olives(pizza) -> float:
    #@wraps()
    def wrapper(*args, **kwargs):
        return pizza()+2.0
    return wrapper

def pizza()->float:
    """Returns the price of a pizza slice """
    return 5.0

print("The price of a pizza slice: ")
print(f"only pizza: {pizza()} ₪")
print(f"pizza + mozzarella: {mozzarella(pizza)()} ₪")
print(f"pizza + olives: {olives(pizza)()} ₪")
print(f"pizza+ onion + mozzarella + olives: {olives(mozzarella(onion(pizza)))()} ₪")

The price of a pizza slice: 
only pizza: 5.0 ₪
pizza + mozzarella: 8.5 ₪
pizza + olives: 7.0 ₪
pizza+ onion + mozzarella + olives: 12.75 ₪
